In [ ]:
import gensim
import os
import collections
import smart_open
import random

In [ ]:
# Set file names for train and test data
#test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
test_data_dir = 'data'
train_file = test_data_dir + os.sep + 'ingredient2vec'
#test_file = test_data_dir + os.sep + 'lee.cor'

In [143]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                line_split = line.split(' ')
                ingredient = line_split[0]
                compounds = ' '.join(line_split[1:])
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(compounds), [ingredient])

In [144]:
train_corpus = list(read_corpus(train_file))
#test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

In [145]:
train_corpus[:20]

[TaggedDocument(words=[u'nonanone'], tags=['ruta_chalepensis_oil']),
 TaggedDocument(words=[u'butyl_butyrate'], tags=['passiflora_mollissima']),
 TaggedDocument(words=[u'thymol', u'octanone', u'pentylfuran', u'nerolidol', u'myrcene', u'formyl', u'ene', u'isopulegol', u'salicylaldehyde', u'mentha', u'dien', u'ol', u'octanal', u'methylanisole', u'veratraldehyde'], tags=['mastic_gum_oil']),
 TaggedDocument(words=[u'benzyl_alcohol'], tags=['copaiba']),
 TaggedDocument(words=[u'bornyl_acetate', u'isoamyl_alcohol'], tags=['teucrium_chamaedrys']),
 TaggedDocument(words=[u'menth', u'ene', u'al', u'piperitone', u'formyl', u'ene'], tags=['seed_oil']),
 TaggedDocument(words=[u'taurine'], tags=['octopus']),
 TaggedDocument(words=[u'phenyl', u'propanol'], tags=['storax']),
 TaggedDocument(words=[u'trimethyl', u'cyclohexa', u'dienyl', u'but', u'en', u'one'], tags=['summer_savory']),
 TaggedDocument(words=[u'diacetyl'], tags=['fagroea_racemosa_flower']),
 TaggedDocument(words=[u'undecalactone', u'hex

In [146]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=5, iter=55)

#dm/m,d50,n5,w5,mc5,s0.001,t3

In [147]:
model.build_vocab(train_corpus)

In [148]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

CPU times: user 4.98 s, sys: 1.85 s, total: 6.84 s
Wall time: 4.12 s


1480710

In [149]:
print model.infer_vector(train_corpus[0].words)

[-0.04531394 -0.04760883  0.05232451 -0.02326259  0.1174707  -0.03494434
 -0.04985017 -0.05959274 -0.11027812 -0.04754625 -0.21017672  0.09437538
 -0.0414009  -0.04445948 -0.00481078  0.04646252 -0.07737248  0.06032601
 -0.06207474  0.14573263  0.06198218  0.0740926  -0.17980683  0.17336413
  0.0474151  -0.01940092 -0.00982774  0.03669292  0.1136224   0.01171486
 -0.15590233  0.05030093 -0.28692093 -0.0865595   0.15751883 -0.02024405
  0.01057846 -0.06774758  0.00336296  0.00458928 -0.02315905 -0.09257079
  0.0327879   0.05289691  0.02327102 -0.12980983 -0.18205382  0.0655232
 -0.06209823 -0.13749428]


In [150]:
print train_corpus[0]

print train_corpus[0].tags, train_corpus[0].words

inferred_vector = model.infer_vector(train_corpus[0].words)
model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))




TaggedDocument([u'nonanone'], ['ruta_chalepensis_oil'])
['ruta_chalepensis_oil'] [u'nonanone']


[('vitis_labrusca_juice', 0.45572900772094727),
 ('berry', 0.45533114671707153),
 ('magnolia_oil', 0.43487441539764404),
 ('lavender', 0.4300282597541809),
 ('plant', 0.38228335976600647),
 ('french_lavender', 0.3763052821159363),
 ('pimento', 0.3728765845298767),
 ('cinchona_officinalis', 0.3658626675605774),
 ('crownberry', 0.35935094952583313),
 ('banana_passion_fruit', 0.3552214801311493),
 ('black_chokecherry', 0.33764782547950745),
 ('wild_bergamot', 0.3352782726287842),
 ('sour_cherry', 0.3341272473335266),
 ('artemisia_herba_alba', 0.33113616704940796),
 ('rabbiteye_blueberry', 0.32571324706077576),
 ('east_indian_geranium', 0.324581116437912),
 ('european_cranberry', 0.3228362798690796),
 ('prune', 0.3211865723133087),
 ('orange_peel_oil', 0.3164363503456116),
 ('salvia_grandifiora_oil', 0.31402838230133057),
 ('kiwi', 0.3132840394973755),
 ('juniper_berry', 0.31067970395088196),
 ('tsuga_canadensis', 0.30916762351989746),
 ('litsea_zeylanica_oil', 0.3086841404438019),
 ('wild

In [151]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(train_corpus[doc_id].tags[0])
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [152]:
collections.Counter(ranks)  # Results vary due to random seeding and very small corpus

Counter({0: 286,
         1: 61,
         2: 29,
         3: 31,
         4: 14,
         5: 17,
         6: 13,
         7: 17,
         8: 13,
         9: 12,
         10: 7,
         11: 9,
         12: 6,
         13: 10,
         14: 9,
         15: 5,
         16: 5,
         17: 6,
         18: 5,
         19: 2,
         20: 7,
         21: 4,
         22: 9,
         23: 5,
         24: 5,
         25: 4,
         26: 3,
         27: 5,
         28: 6,
         29: 2,
         30: 1,
         31: 6,
         32: 2,
         33: 1,
         34: 2,
         35: 3,
         36: 4,
         37: 2,
         38: 4,
         39: 5,
         40: 4,
         41: 7,
         42: 4,
         43: 6,
         44: 3,
         45: 5,
         46: 4,
         47: 3,
         48: 3,
         49: 7,
         50: 4,
         51: 4,
         52: 4,
         53: 3,
         54: 4,
         55: 3,
         56: 1,
         57: 6,
         58: 5,
         59: 5,
         60: 3,
         61: 4,
      

In [153]:
print second_ranks

[('grapefruit_peel', 0.8154087066650391), ('hatsutake', 0.8967252969741821), ('juniperus_sabina', 0.642683744430542), ('phallus_impudicus', 0.8495807647705078), ('artemisia_camphorata', 0.790740966796875), ('strawberry_juice', 0.9376745223999023), ('orange_flower', 0.8781540989875793), ('sumatra_benzoin', 0.9297366738319397), ('calytrix_tetragona_oil', 0.9312870502471924), ('tulip', 0.8322804570198059), ('chamaecyparis_pisifera_oil', 0.8398119211196899), ('russian_star_anise', 0.8006571531295776), ('sweetfish', 0.8361461162567139), ('blackberry_juice', 0.870078444480896), ('labiatae', 0.7813615798950195), ('chinese_quince_peel', 0.28108683228492737), ('polianthes_tuberosa_oil', 0.8778105974197388), ('calyptranthes_parriculata', 0.9123133420944214), ('litsea_zeylanica', 0.7306162118911743), ('parsnip_fruit', 0.8456422090530396), ('oregano_oil', 0.8875131607055664), ('magnolia_grandiflora', 0.6648386716842651), ('chinese_quince_peel', 0.28108683228492737), ('artemisia_scoparia', 0.847673

In [156]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(train_corpus))

# Compare and print the most/median/least similar documents from the train corpus
print('Train Document ({}, {}): [{}]\n'.format(doc_id, train_corpus[doc_id].tags[0], ' '.join(train_corpus[doc_id].words)))
inferred_vector = model.infer_vector(train_corpus[doc_id].words)
print inferred_vector

sims = model.docvecs.most_similar([inferred_vector], topn=10)
for sim in sims:
    print sim

Train Document (389, cassia_oil): [salicylaldehyde cresol]

[ 0.00560218 -0.03508722  0.06224342 -0.04316341 -0.00800517 -0.07957029
  0.02935229 -0.07251674 -0.11426245  0.10925957 -0.2246262   0.06100965
 -0.05865174 -0.09546181 -0.10639692  0.14534409  0.08480853 -0.05602359
 -0.18121827  0.12102835  0.02057474  0.10029595 -0.06595494  0.17731512
  0.05975069 -0.14628707 -0.0104727  -0.0229037  -0.01054172  0.03628368
 -0.11294177  0.00203785 -0.05777577 -0.00309495  0.09032778  0.02389644
 -0.05602477  0.06858458 -0.163249   -0.04896591  0.02012918 -0.07774185
 -0.01027591  0.10736061 -0.02807609 -0.105499   -0.03286669 -0.03953042
 -0.07733793 -0.18651368]
('cassia_oil', 0.793540358543396)
('crepis_foetida_root', 0.791661262512207)
('pinus_avium_fruit', 0.7882527112960815)
('ceanothus_velutinus_leaf', 0.7866560816764832)
('cinnamomum_oil', 0.7775654792785645)
('anise_seed', 0.7755776643753052)
('labdanum', 0.7722835540771484)
('orthodon', 0.7562825679779053)
('elsholtzia_argyi', 0

In [157]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(train_corpus))
inferred_vector = model.infer_vector(train_corpus[doc_id].words)
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Train Document ({}, {}): [{}]\n'.format(doc_id, train_corpus[doc_id].tags[0], ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print label, sims[index]
    #print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Train Document (673, kidney_bean): [nonadienal jasmone lauric_acid pentanone hexyl_acetate methylthio octanol methoxy vinylphenol nonanoic_acid maltol octanone methyl_sulfide propyl_alcohol penten one pentanedione nonalactone trimethyl cyclohexa dienyl but en one mentha diene nonenal decanoic_acid octadecenal methyl butanol hexalactone vinylphenol hexanol heptanone octanal hexanoic_acid lauryl_alcohol lysine methyl buten ol ionone ethyl_alcohol acetylthiazole methyl thiazoleethanol vanillin alpha terpineol isobutyl methoxypyrazine octanone pentanol methoxy methylpropyl pyrazine nerolidol heptanoic_acid methoxy propionaldehyde decenal heptanol theobromine butyrolactone octen ol hexen ol heptenal palmitic_acid hexen ol phenol penten ol octanoic_acid hexyl_alcohol nonyl_alcohol ionone mentha diene nonanal valeraldehyde myristic_acid acetylpyridine one guaiacol]

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc5,s0.001,t3):

MOST ('prawn', 0.9289549589157104)
MEDIAN ('manila_cop

In [171]:
print train_corpus[0].tags

['ruta_chalepensis_oil']
